In [5]:
import pandas as pd
from sklearn.linear_model import Ridge, RidgeClassifier
from sklearn.model_selection import train_test_split
from numpy import linspace
from IPython.display import Image, display
from difflib import SequenceMatcher
##pandasprofiling

In [58]:
df =pd.read_csv("data/ejemplo_peliculas.csv")

# EDA

In [11]:
df.head()

,fecha,dist,ranking,rating,title,week,admisiones
0,2017-04-06,FOXI,1.0,A,Un Jefe en Pañales,1.0,209688.0
1,2017-04-06,VIDEOC,2.0,B,3 Idiotas,1.0,91685.0
2,2017-04-06,PPI,3.0,B,La Vigilante del Futuro,1.0,65765.0
3,2017-04-06,SPE,4.0,AA,Los Pitufos en la Aldea Perdida,0.0,53172.0
4,2017-04-06,DISNEY,5.0,A,La Bella y la Bestia,3.0,44254.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187360 entries, 0 to 187359
Data columns (total 7 columns):
fecha         187360 non-null object
dist          187360 non-null object
ranking       187360 non-null float64
rating        164291 non-null object
title         187360 non-null object
week          187360 non-null float64
admisiones    187360 non-null float64
dtypes: float64(3), object(4)
memory usage: 10.0+ MB


In [8]:
df.describe()

,ranking,week,admisiones
count,187360.000000,187360.000000,1.873600e+05
mean,40.387452,9.086571,8.876716e+03
std,23.440656,16.052126,5.879185e+04
min,1.000000,-158.000000,0.000000e+00
25%,22.000000,2.000000,1.000000e+00
50%,39.000000,5.000000,6.000000e+01
75%,59.000000,9.000000,5.660000e+02
max,128.000000,526.000000,3.792669e+06


In [9]:
df.tail()

,fecha,dist,ranking,rating,title,week,admisiones
187355,2016-02-29,CORA,72.0,B-15,Lado Oscuro del Deseo,25.0,0.0
187356,2016-02-29,INDP,72.0,A,Lucifer,5.0,0.0
187357,2016-02-29,CONTCNPL,72.0,B,SM TOWN The Stage,0.0,0.0
187358,2016-02-29,CONTCNPL,72.0,B,Imagine Dragons: Smoke + Mirrors Live,0.0,0.0
187359,2016-02-29,TONALADIST,72.0,C,Hide and Seek A las Escondidas,4.0,0.0


- Queremos predecir a partir del nombre de las películas

In [12]:
df.rating.value_counts()

B       67881
B-15    52310
A       30040
C        9032
AA       4757
D         271
Name: rating, dtype: int64

In [17]:
df.rating.value_counts(dropna=False)

B       67881
B-15    52310
A       30040
NaN     23069
C        9032
AA       4757
D         271
Name: rating, dtype: int64

In [18]:
df[df["rating"].isnull()]

,fecha,dist,ranking,rating,title,week,admisiones
15,2017-04-06,CONTCNPL,16.0,NaN,2017 Contenido Alternativo,14.0,1186.0
24,2017-04-06,FYM,25.0,NaN,2017 Festival Ambulante,2.0,732.0
25,2017-04-06,CONTCNPL,26.0,NaN,Cine de Aventura Seleccion Freeman 2017,10.0,315.0
27,2017-04-06,FYM,28.0,NaN,Retrospectiva Stanley Kubrick 2017,14.0,295.0
33,2017-04-06,FYM,34.0,NaN,61 Muestra Internacional de la Cineteca,21.0,124.0
34,2017-04-06,FYM,35.0,NaN,2017 Festivales y Muestras,14.0,112.0
38,2017-04-06,FYM,39.0,NaN,Semana de Cine Brasileño 2017,1.0,56.0
75,2017-04-06,FYM,65.0,NaN,2017 Clasicos en Pantalla Grande,14.0,0.0
77,2017-04-06,FYM,65.0,NaN,Matinee Infantil de la Cineteca 2017,14.0,0.0
108,2017-12-08,FYM,30.0,NaN,2017 Festivales y Muestras,50.0,434.0


- Se decide tirar los null (na) porque pareciera que la mayoría corresonde a festivales y cosas raras que no son películas

- Convertir a minúsculas los títulos

In [59]:
df["title"]=df["title"].str.lower()

- Quitar duplicados de nombres de películas

In [60]:
df=df.drop_duplicates("title")

- Quitar nulos en rating

In [61]:
df=df[df["rating"].notnull()]

In [62]:
len(df)

2053

In [27]:
df

,fecha,dist,ranking,rating,title,week,admisiones
0,2017-04-06,FOXI,1.0,A,un jefe en pañales,1.0,209688.0
1,2017-04-06,VIDEOC,2.0,B,3 idiotas,1.0,91685.0
2,2017-04-06,PPI,3.0,B,la vigilante del futuro,1.0,65765.0
3,2017-04-06,SPE,4.0,AA,los pitufos en la aldea perdida,0.0,53172.0
4,2017-04-06,DISNEY,5.0,A,la bella y la bestia,3.0,44254.0
5,2017-04-06,VIDEOC,6.0,B,power rangers,2.0,22122.0
6,2017-04-06,CORA,7.0,B,la cabaña,0.0,16964.0
7,2017-04-06,WBI,8.0,B-15,chips patrulla motorizada,0.0,13962.0
8,2017-04-06,CNPLS,9.0,B-15,la posesion de altair,0.0,12148.0
9,2017-04-06,SPE,10.0,B,life vida inteligente,2.0,10060.0


# Convertir el rating en algo predecible

In [63]:
df[df["rating"]=="D"]

,fecha,dist,ranking,rating,title,week,admisiones
35,2017-04-06,PIANO,36.0,D,tenemos la carne,2.0,103.0
842,2016-01-24,CANIBAL,81.0,D,love,7.0,100.0
1031,2016-11-11,CNAUTA,74.0,D,atroz,-102.0,43.0


In [64]:
converter = {"AA": 6, "A": 9, "B": 12, "B-15": 15, "C": 18, "D": 21}

In [65]:
df["rating"].replace(converter)

0          9
1         12
2         12
3          6
4          9
5         12
6         12
7         15
8         15
9         12
10        12
11        18
12        12
13        15
14        15
16        12
17        18
18        15
19        12
20        12
21        12
22        18
23        15
26        12
28        12
29        12
30        15
31        15
32        15
35        21
          ..
38374     18
38394      9
38565     12
39789      9
39796     15
40181     12
40189      9
40245     12
40248     12
44109     15
44112     12
44856     18
48068      9
51458     15
54470     18
54474     12
57856      9
57885      6
64499      9
64508      6
64510     15
65244     15
68835     12
68836     12
69938     15
78730      9
109735    15
109742     9
109744     9
109750    15
Name: rating, Length: 2053, dtype: int64

In [66]:
df["rating_continuous"]=df["rating"].replace(converter)

In [67]:
df

,fecha,dist,ranking,rating,title,week,admisiones,rating_continuous
0,2017-04-06,FOXI,1.0,A,un jefe en pañales,1.0,209688.0,9
1,2017-04-06,VIDEOC,2.0,B,3 idiotas,1.0,91685.0,12
2,2017-04-06,PPI,3.0,B,la vigilante del futuro,1.0,65765.0,12
3,2017-04-06,SPE,4.0,AA,los pitufos en la aldea perdida,0.0,53172.0,6
4,2017-04-06,DISNEY,5.0,A,la bella y la bestia,3.0,44254.0,9
5,2017-04-06,VIDEOC,6.0,B,power rangers,2.0,22122.0,12
6,2017-04-06,CORA,7.0,B,la cabaña,0.0,16964.0,12
7,2017-04-06,WBI,8.0,B-15,chips patrulla motorizada,0.0,13962.0,15
8,2017-04-06,CNPLS,9.0,B-15,la posesion de altair,0.0,12148.0,15
9,2017-04-06,SPE,10.0,B,life vida inteligente,2.0,10060.0,12


# Partir títulos en lista de palabras

In [70]:
df["splitted"]=df["title"].str.split(" ")
df

,fecha,dist,ranking,rating,title,week,admisiones,rating_continuous,splitted
0,2017-04-06,FOXI,1.0,A,un jefe en pañales,1.0,209688.0,9,"[un, jefe, en, pañales]"
1,2017-04-06,VIDEOC,2.0,B,3 idiotas,1.0,91685.0,12,"[3, idiotas]"
2,2017-04-06,PPI,3.0,B,la vigilante del futuro,1.0,65765.0,12,"[la, vigilante, del, futuro]"
3,2017-04-06,SPE,4.0,AA,los pitufos en la aldea perdida,0.0,53172.0,6,"[los, pitufos, en, la, aldea, perdida]"
4,2017-04-06,DISNEY,5.0,A,la bella y la bestia,3.0,44254.0,9,"[la, bella, y, la, bestia]"
5,2017-04-06,VIDEOC,6.0,B,power rangers,2.0,22122.0,12,"[power, rangers]"
6,2017-04-06,CORA,7.0,B,la cabaña,0.0,16964.0,12,"[la, cabaña]"
7,2017-04-06,WBI,8.0,B-15,chips patrulla motorizada,0.0,13962.0,15,"[chips, patrulla, motorizada]"
8,2017-04-06,CNPLS,9.0,B-15,la posesion de altair,0.0,12148.0,15,"[la, posesion, de, altair]"
9,2017-04-06,SPE,10.0,B,life vida inteligente,2.0,10060.0,12,"[life, vida, inteligente]"


In [71]:
big_list = []
for movie_name in df["splitted"]:
    big_list += movie_name

In [72]:
big_series=pd.Series(big_list).value_counts()

In [73]:
(big_series >=2).sum()

800

In [74]:
at_least_once = big_series[big_series >= 2]

# Data prep final

In [75]:
df["splitted_filtered"] = df["splitted"].apply(lambda x: [k for k in x if k in at_least_once])

In [43]:
df["splitted_filtered"].apply(lambda x: len(x) > 0)

0          True
1          True
2          True
3          True
4          True
5         False
6          True
7         False
8          True
9          True
10         True
11         True
12         True
13         True
14         True
16         True
17         True
18         True
19        False
20        False
21        False
22         True
23        False
26         True
28         True
29         True
30         True
31        False
32        False
35         True
          ...  
38374     False
38394      True
38565      True
39789      True
39796      True
40181      True
40189      True
40245      True
40248      True
44109      True
44112      True
44856      True
48068      True
51458      True
54470      True
54474      True
57856      True
57885      True
64499      True
64508      True
64510      True
65244      True
68835      True
68836      True
69938      True
78730     False
109735     True
109742     True
109744     True
109750     True
Name: splitted_filtered,

In [76]:
df = df[df["splitted_filtered"].apply(lambda x: len(x) > 0)]

In [45]:
df

,fecha,dist,ranking,rating,title,week,admisiones,rating_continuous,splitted,splitted_filtered
0,2017-04-06,FOXI,1.0,A,un jefe en pañales,1.0,209688.0,9,"[un, jefe, en, pañales]","[un, jefe, en]"
1,2017-04-06,VIDEOC,2.0,B,3 idiotas,1.0,91685.0,12,"[3, idiotas]",[3]
2,2017-04-06,PPI,3.0,B,la vigilante del futuro,1.0,65765.0,12,"[la, vigilante, del, futuro]","[la, vigilante, del, futuro]"
3,2017-04-06,SPE,4.0,AA,los pitufos en la aldea perdida,0.0,53172.0,6,"[los, pitufos, en, la, aldea, perdida]","[los, en, la, perdida]"
4,2017-04-06,DISNEY,5.0,A,la bella y la bestia,3.0,44254.0,9,"[la, bella, y, la, bestia]","[la, bella, y, la, bestia]"
6,2017-04-06,CORA,7.0,B,la cabaña,0.0,16964.0,12,"[la, cabaña]","[la, cabaña]"
8,2017-04-06,CNPLS,9.0,B-15,la posesion de altair,0.0,12148.0,15,"[la, posesion, de, altair]","[la, posesion, de]"
9,2017-04-06,SPE,10.0,B,life vida inteligente,2.0,10060.0,12,"[life, vida, inteligente]","[life, vida]"
10,2017-04-06,WBI,11.0,B,kong la isla calavera,4.0,7830.0,12,"[kong, la, isla, calavera]","[la, isla, calavera]"
11,2017-04-06,SPE,12.0,C,t2 trainspotting la vida en el abismo,3.0,3696.0,18,"[t2, trainspotting, la, vida, en, el, abismo]","[trainspotting, la, vida, en, el, abismo]"


In [77]:
at_least_once

la            395
de            364
el            351
del           133
en            109
los           100
un             93
y              86
a              72
las            61
una            55
mi             51
no             47
2              37
que            36
por            33
al             30
vida           29
the            28
amor           26
con            23
sin            22
es             22
mas            21
hombre         20
gran           20
casa           20
yo             19
mundo          19
noche          19
             ... 
piedra          2
hogar           2
pie             2
reyes           2
batalla         2
altura          2
orgullo         2
largo           2
mono            2
mike            2
big             2
reloj           2
filtro          2
obsesion        2
quien           2
estaras         2
maligno         2
calavera        2
huerfanos       2
amityville      2
soledad         2
kinder          2
ideal           2
angry           2
irak      

In [78]:
for col in at_least_once.index:
    df[col] = df["splitted_filtered"].apply(lambda x: True if col in x else False)

/Users/jalfredomb/.pyenv/versions/3.5.0/lib/python3.5/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [79]:
df

,fecha,dist,ranking,rating,title,week,admisiones,rating_continuous,splitted,splitted_filtered,...,soledad,kinder,ideal,angry,irak,steel,campeones,apuesta,mueren,inicio
0,2017-04-06,FOXI,1.0,A,un jefe en pañales,1.0,209688.0,9,"[un, jefe, en, pañales]","[un, jefe, en]",...,False,False,False,False,False,False,False,False,False,False
1,2017-04-06,VIDEOC,2.0,B,3 idiotas,1.0,91685.0,12,"[3, idiotas]",[3],...,False,False,False,False,False,False,False,False,False,False
2,2017-04-06,PPI,3.0,B,la vigilante del futuro,1.0,65765.0,12,"[la, vigilante, del, futuro]","[la, vigilante, del, futuro]",...,False,False,False,False,False,False,False,False,False,False
3,2017-04-06,SPE,4.0,AA,los pitufos en la aldea perdida,0.0,53172.0,6,"[los, pitufos, en, la, aldea, perdida]","[los, en, la, perdida]",...,False,False,False,False,False,False,False,False,False,False
4,2017-04-06,DISNEY,5.0,A,la bella y la bestia,3.0,44254.0,9,"[la, bella, y, la, bestia]","[la, bella, y, la, bestia]",...,False,False,False,False,False,False,False,False,False,False
6,2017-04-06,CORA,7.0,B,la cabaña,0.0,16964.0,12,"[la, cabaña]","[la, cabaña]",...,False,False,False,False,False,False,False,False,False,False
8,2017-04-06,CNPLS,9.0,B-15,la posesion de altair,0.0,12148.0,15,"[la, posesion, de, altair]","[la, posesion, de]",...,False,False,False,False,False,False,False,False,False,False
9,2017-04-06,SPE,10.0,B,life vida inteligente,2.0,10060.0,12,"[life, vida, inteligente]","[life, vida]",...,False,False,False,False,False,False,False,False,False,False
10,2017-04-06,WBI,11.0,B,kong la isla calavera,4.0,7830.0,12,"[kong, la, isla, calavera]","[la, isla, calavera]",...,False,False,False,False,False,False,False,False,False,False
11,2017-04-06,SPE,12.0,C,t2 trainspotting la vida en el abismo,3.0,3696.0,18,"[t2, trainspotting, la, vida, en, el, abismo]","[trainspotting, la, vida, en, el, abismo]",...,False,False,False,False,False,False,False,False,False,False


# Partir data

In [80]:
train, test = train_test_split(df, random_state=6)

In [81]:
rc = Ridge(alpha=1)

In [85]:
rc.fit(X=train[at_least_once.index],y=train["rating_continuous"])

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None, normalize=False,
      random_state=None, solver='auto', tol=0.001)

- zip: junta dos listas

In [86]:
coef_eval=pd.DataFrame(zip(at_least_once.index,rc.coef_.tolist()),columns=["Palabra","Coeficiente"])

TypeError: data argument can't be an iterator